In [2]:
%load_ext autoreload
%autoreload 2

import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
import plotly.subplots as sp

df = pd.DataFrame
sns.set_theme()

In [3]:
w,x,wxb = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 7,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)

[STIM_GEN] Generating random weights


In [10]:
rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtl_bipolar.txt')
df(rtl)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-22.0,-3.0,-71.0,1.0,57.0,0.0,99.0,0.0,97.0,-2.0,...,19.0,3.0,121.0,3.0,125.0,2.0,123.0,-1.0,-91.0,-2.0
1,121.0,3.0,35.0,-1.0,46.0,3.0,-102.0,-3.0,-44.0,2.0,...,38.0,1.0,-30.0,-3.0,-56.0,1.0,121.0,3.0,-43.0,2.0
2,37.0,1.0,-69.0,-3.0,-121.0,-4.0,-21.0,2.0,69.0,-3.0,...,25.0,0.0,99.0,0.0,97.0,0.0,63.0,2.0,-55.0,-2.0
3,-52.0,0.0,66.0,-2.0,32.0,-3.0,-14.0,-2.0,-39.0,0.0,...,-6.0,0.0,-3.0,-1.0,-38.0,-1.0,14.0,2.0,-44.0,2.0
4,-95.0,2.0,17.0,-3.0,-69.0,0.0,2.0,-2.0,59.0,1.0,...,-83.0,-2.0,107.0,-3.0,-9.0,1.0,109.0,2.0,73.0,-1.0
5,17.0,2.0,-16.0,-2.0,-126.0,-1.0,18.0,0.0,-64.0,0.0,...,-31.0,1.0,74.0,2.0,28.0,1.0,-51.0,1.0,-115.0,2.0
6,-84.0,2.0,12.0,-4.0,-77.0,2.0,98.0,-2.0,80.0,-2.0,...,-28.0,2.0,107.0,0.0,-8.0,1.0,83.0,1.0,-62.0,2.0
7,-23.0,2.0,-63.0,1.0,-23.0,2.0,89.0,3.0,53.0,-4.0,...,41.0,0.0,-123.0,1.0,101.0,1.0,-53.0,0.0,-63.0,-1.0
8,-99.0,2.0,105.0,3.0,-99.0,-1.0,-96.0,-4.0,12.0,-4.0,...,-37.0,-3.0,-61.0,-1.0,36.0,-4.0,13.0,-4.0,61.0,-3.0
9,-58.0,-2.0,8.0,-1.0,-46.0,0.0,-40.0,0.0,56.0,0.0,...,46.0,0.0,-84.0,-1.0,-92.0,0.0,-120.0,-4.0,38.0,-3.0


In [10]:
x[0]

array([-123,  124,  -48,    0, -106,   -6,  -12,  -25, -117, -111, -125,
        -81,   62,   47,  -17,  -95,  104,   24,   62,    2,  -91,  -38,
       -119,  -80,  -36, -127,   36, -101,   63,  -16,   53,  107,   23,
         32,    2,  -34, -124, -124,  -29,  -11,   37,   43,  -17, -107,
        106,    6,   89,  -44,   55,  102, -124,  -44,   24,  -79,  -33,
         31,   90,    8,   43,   13, -123,  113,   28,   77,  -16,   75,
        125,   44,   54,  111,   11,  116,  -84, -121,   72, -127,  -76,
         35,  -31, -104,   65,   72,   31, -109,  -95, -125,   39, -115,
        -63,   -1,  125,  -55, -124,   14,   -1,   91,    4,  103,   71,
          7,   21,  -69, -125,   86,   48,   53,  -69,  -17,  -45,  127,
        -47,  -70,   44,  -94,  107,  -58,   94,  -48,  -38,  110,  118,
       -103,  -27,   61,   -6,  -63,   62,  -50])

In [11]:
q.simulate_trit_mul(w, x[1], 7, verbose=True)

x trits:
     0  1  2  3  4  5  6
0   -1 -1 -1  0  0 -1 -1
1    1  0  0  0  1  1  1
2    0  0  0 -1 -1 -1  0
3   -1  0  0 -1 -1 -1  0
4    0  0  0  0  1  0  0
..  .. .. .. .. .. .. ..
123  1  0  0  0  1  1  0
124  0  1  0  1  0  1  0
125 -1 -1  0  0  0 -1 -1
126  1  1  1  0  1  1  1
127  0  0  0  0 -1  0 -1

[128 rows x 7 columns]
partials:
   0   1   2   3   4   5   6   7   8   9   ...  22  23  24  25  26  27  28  \
0   7  -1  17 -11   9   7   5 -15  -7  -3  ... -13   3  -1 -11   1   1   7   
1  13  -5  11  -7  13   9  -3  -3   1 -11  ...   7  11  -1  -5 -11  -3  -3   
2  17   3   5   5  -3  -1   5 -13  -1   1  ...  -7   5  -1 -13  -3  -1   3   
3  10  -4   4  12  10   2  18   2  -8   0  ...  10  18   0  -4  -8  -6  -4   
4  10  -2  14  -6   8   0   0  -8 -12  -6  ...  -8  -2  -4 -18  -2   6   4   
5  14  -6   2   2   6   4   8  -6   2  -4  ...   2   6  -8 -14   2   6  -6   
6  10  10  -2   6   4  12  10  -6   4  -6  ...   4   8   0  -4   0   4   2   

   29  30  31  
0  15  -3   3  


array([  588.,   472.,  2166.,   374.,   -56.,  -404., -1208.,  -144.,
         780.,  -664.,   -94.,  -942.,  -768.,   192., -1240.,   272.,
        1236.,  1922.,   584.,  -554.,   508.,  -586.,  -566.,  -536.,
       -1298.,   474.,   756.,  1072.,  -766.,  1610.,  -218.,  1294.])

In [8]:
x @ w.T

array([[ -995,   779,  -475,  -995,   321,  -443,   531,  -215,   963,
         -425,  1387,   -63,   175,   541,  -327,  1223,  -355,    -1,
          637,  2093,  1595,   175,  -981,   751,  -607,  -707,  1895,
         -425,    99,    57,   765,  -611],
       [  588,   472,  2166,   374,   -56,  -404, -1208,  -144,   780,
         -664,   -94,  -942,  -768,   192, -1240,   272,  1236,  1922,
          584,  -554,   508,  -586,  -566,  -536, -1298,   474,   756,
         1072,  -766,  1610,  -218,  1294],
       [-1373,   521,   991, -1109,  -339,  1141,  -555,  -167,    73,
          -71, -1267,  -411,  1013,  -249, -1385,   391,  -319,   703,
           17,   139,    25,   885,  -525,   813,   399,  -423,   581,
        -1035,  1005,  -905,  -701,   299],
       [-1354,  -410,  1474,   826,  1228,  -750,   700,  -614,  -724,
          592,   648,   214,   478,  -518,   754,  1028,  1640,   846,
          -38,    90,   276,  -160,  -372,  -278,  -474,   292, -1200,
          220,  